# GlobalMart Sales Data - Exploratory Data Analysis

## Part 1: Beginner Level - The Visual Creator

**Focus:** Data cleaning, analysis, and visualization - mastering the foundations of the data science workflow.

### Task 1: Crafting the First Cut (Basic Editing Workflow)

**Description:** Perform an in-depth Exploratory Data Analysis (EDA) on GlobalMart's customer purchase dataset

---

## 1. Load and Inspect Data

Import necessary libraries and load the dataset to check for missing values, data types, and outliers.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Configure visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('/mnt/user-data/uploads/supermarket_sales.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\n" + "="*60)
print("First 5 Rows:")
print("="*60)
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
print("="*60)
df.info()

In [ ]:
# Statistical summary
print("Statistical Summary:")
print("="*60)
df.describe()

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("="*60)
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found in the dataset!")

In [ ]:
# Check for duplicate rows
print("Duplicate Records Analysis:")
print("="*60)
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    print(f"\nDuplicate percentage: {(duplicate_count/len(df))*100:.2f}%")
else:
    print("No duplicate rows found!")

## 2. Clean the Data

Handle missing values, duplicates, and inconsistent formatting.

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Convert Order_Date to datetime
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'])

# Extract additional date features
df_clean['Year'] = df_clean['Order_Date'].dt.year
df_clean['Month'] = df_clean['Order_Date'].dt.month
df_clean['Month_Name'] = df_clean['Order_Date'].dt.strftime('%B')
df_clean['Quarter'] = df_clean['Order_Date'].dt.quarter
df_clean['Day_of_Week'] = df_clean['Order_Date'].dt.day_name()

# Calculate success rate (Won deals)
df_clean['Is_Won'] = (df_clean['Stage'] == 'Won').astype(int)

# Calculate achievement rate
df_clean['Achievement_Rate'] = np.where(
    df_clean['Target'] > 0,
    (df_clean['Revenue'] / df_clean['Target']) * 100,
    0
)

print("Data cleaning completed!")
print(f"\nCleaned dataset shape: {df_clean.shape}")
print("\nNew columns added:")
print("- Year, Month, Month_Name, Quarter, Day_of_Week")
print("- Is_Won (binary indicator for won deals)")
print("- Achievement_Rate (Revenue/Target percentage)")

In [ ]:
# Display cleaned data sample
print("Sample of cleaned data with new features:")
df_clean[['Order_Date', 'Year', 'Month_Name', 'Quarter', 'Day_of_Week', 
          'Revenue', 'Target', 'Achievement_Rate', 'Is_Won']].head(10)

## 3. Analyze and Visualize

Identify top-selling products, cities, and customer demographics. Create comprehensive visualizations.

### 3.1 Sales Performance Overview

In [ ]:
# Overall metrics
print("GLOBALMART SALES PERFORMANCE SUMMARY")
print("="*60)
print(f"Total Orders: {len(df_clean):,}")
print(f"Total Revenue: ${df_clean['Revenue'].sum():,.2f}")
print(f"Total Target: ${df_clean['Target'].sum():,.2f}")
print(f"Total Units Sold: {df_clean['Units_Sold'].sum():,}")
print(f"\nAverage Revenue per Order: ${df_clean['Revenue'].mean():,.2f}")
print(f"Average Deal Size: ${df_clean['Deal_Size'].mean():,.2f}")
print(f"\nWon Deals: {df_clean[df_clean['Stage']=='Won'].shape[0]:,}")
print(f"Lost Deals: {df_clean[df_clean['Stage']=='Lost'].shape[0]:,}")
print(f"Opportunities: {df_clean[df_clean['Stage']=='Opportunity'].shape[0]:,}")
print(f"\nWin Rate: {(df_clean['Is_Won'].sum()/len(df_clean))*100:.2f}%")

In [ ]:
# Sales performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sales Performance Overview', fontsize=16, fontweight='bold', y=0.995)

# 1. Deal Stage Distribution
stage_counts = df_clean['Stage'].value_counts()
colors_stage = ['#2ecc71', '#e74c3c', '#f39c12']
axes[0, 0].pie(stage_counts.values, labels=stage_counts.index, autopct='%1.1f%%',
               colors=colors_stage, startangle=90)
axes[0, 0].set_title('Deal Stage Distribution', fontsize=12, fontweight='bold')

# 2. Revenue vs Target by Stage
stage_revenue = df_clean.groupby('Stage')[['Revenue', 'Target']].sum()
stage_revenue.plot(kind='bar', ax=axes[0, 1], color=['#3498db', '#e67e22'])
axes[0, 1].set_title('Revenue vs Target by Deal Stage', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Deal Stage')
axes[0, 1].set_ylabel('Amount ($)')
axes[0, 1].legend(['Revenue', 'Target'])
axes[0, 1].tick_params(axis='x', rotation=0)

# 3. Monthly Revenue Trend
monthly_revenue = df_clean.groupby('Month')['Revenue'].sum()
axes[1, 0].plot(monthly_revenue.index, monthly_revenue.values, marker='o', 
                linewidth=2, markersize=8, color='#9b59b6')
axes[1, 0].fill_between(monthly_revenue.index, monthly_revenue.values, alpha=0.3, color='#9b59b6')
axes[1, 0].set_title('Monthly Revenue Trend', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Revenue ($)')
axes[1, 0].grid(True, alpha=0.3)

# 4. Units Sold Distribution
won_deals = df_clean[df_clean['Stage'] == 'Won']
axes[1, 1].hist(won_deals['Units_Sold'], bins=20, color='#1abc9c', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Distribution of Units Sold (Won Deals)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Units Sold')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].axvline(won_deals['Units_Sold'].mean(), color='red', linestyle='--', 
                   linewidth=2, label=f"Mean: {won_deals['Units_Sold'].mean():.1f}")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 3.2 Product Analysis

In [ ]:
# Product category analysis
print("PRODUCT CATEGORY ANALYSIS")
print("="*60)

category_analysis = df_clean.groupby('Product_Category').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count',
    'Is_Won': 'sum'
}).round(2)

category_analysis.columns = ['Total Revenue', 'Units Sold', 'Total Orders', 'Won Deals']
category_analysis['Win Rate %'] = (category_analysis['Won Deals'] / category_analysis['Total Orders'] * 100).round(2)
category_analysis = category_analysis.sort_values('Total Revenue', ascending=False)

print(category_analysis)

In [ ]:
# Top products analysis
print("\nTOP 5 PRODUCTS BY REVENUE")
print("="*60)

top_products = df_clean.groupby('Product_Name').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count'
}).round(2)

top_products.columns = ['Total Revenue', 'Units Sold', 'Total Orders']
top_products = top_products.sort_values('Total Revenue', ascending=False).head()

print(top_products)

In [ ]:
# Product visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Product Performance Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Revenue by Product Category
category_revenue = df_clean.groupby('Product_Category')['Revenue'].sum().sort_values(ascending=True)
category_revenue.plot(kind='barh', ax=axes[0, 0], color='#e74c3c')
axes[0, 0].set_title('Total Revenue by Product Category', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Revenue ($)')
axes[0, 0].set_ylabel('Product Category')

# 2. Units Sold by Product Category
category_units = df_clean.groupby('Product_Category')['Units_Sold'].sum().sort_values(ascending=False)
axes[0, 1].bar(category_units.index, category_units.values, color='#3498db', edgecolor='black')
axes[0, 1].set_title('Units Sold by Product Category', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Product Category')
axes[0, 1].set_ylabel('Units Sold')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Top 10 Products by Revenue
top_10_products = df_clean.groupby('Product_Name')['Revenue'].sum().nlargest(10).sort_values(ascending=True)
top_10_products.plot(kind='barh', ax=axes[1, 0], color='#2ecc71')
axes[1, 0].set_title('Top 10 Products by Revenue', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Revenue ($)')
axes[1, 0].set_ylabel('Product')

# 4. Product Category Win Rate
category_win_rate = df_clean.groupby('Product_Category').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
category_win_rate['Win Rate'] = (category_win_rate['Is_Won'] / category_win_rate['Order_ID'] * 100)
category_win_rate['Win Rate'].sort_values(ascending=False).plot(kind='bar', ax=axes[1, 1], 
                                                                  color='#f39c12', edgecolor='black')
axes[1, 1].set_title('Win Rate by Product Category', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Product Category')
axes[1, 1].set_ylabel('Win Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].axhline(y=df_clean['Is_Won'].mean()*100, color='red', linestyle='--', 
                   linewidth=2, label='Overall Avg')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 3.3 Geographic Analysis

In [ ]:
# Regional analysis
print("REGIONAL PERFORMANCE ANALYSIS")
print("="*60)

regional_analysis = df_clean.groupby('Region').agg({
    'Revenue': 'sum',
    'Target': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count',
    'Is_Won': 'sum'
}).round(2)

regional_analysis.columns = ['Total Revenue', 'Total Target', 'Units Sold', 'Total Orders', 'Won Deals']
regional_analysis['Win Rate %'] = (regional_analysis['Won Deals'] / regional_analysis['Total Orders'] * 100).round(2)
regional_analysis['Achievement %'] = (regional_analysis['Total Revenue'] / regional_analysis['Total Target'] * 100).round(2)
regional_analysis = regional_analysis.sort_values('Total Revenue', ascending=False)

print(regional_analysis)

In [ ]:
# Country analysis
print("\nTOP 10 COUNTRIES BY REVENUE")
print("="*60)

country_analysis = df_clean.groupby('Country').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count'
}).round(2)

country_analysis.columns = ['Total Revenue', 'Units Sold', 'Total Orders']
country_analysis = country_analysis.sort_values('Total Revenue', ascending=False).head(10)

print(country_analysis)

In [ ]:
# Geographic visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Geographic Performance Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Revenue by Region
region_revenue = df_clean.groupby('Region')['Revenue'].sum().sort_values(ascending=False)
colors_region = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']
axes[0, 0].bar(region_revenue.index, region_revenue.values, color=colors_region, edgecolor='black')
axes[0, 0].set_title('Total Revenue by Region', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Region')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Top 10 Countries by Revenue
top_countries = df_clean.groupby('Country')['Revenue'].sum().nlargest(10).sort_values(ascending=True)
top_countries.plot(kind='barh', ax=axes[0, 1], color='#9b59b6')
axes[0, 1].set_title('Top 10 Countries by Revenue', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Revenue ($)')
axes[0, 1].set_ylabel('Country')

# 3. Regional Market Share
region_share = df_clean.groupby('Region')['Revenue'].sum()
axes[1, 0].pie(region_share.values, labels=region_share.index, autopct='%1.1f%%',
               colors=colors_region, startangle=90)
axes[1, 0].set_title('Regional Market Share by Revenue', fontsize=12, fontweight='bold')

# 4. Regional Win Rate Comparison
region_win = df_clean.groupby('Region').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
region_win['Win Rate'] = (region_win['Is_Won'] / region_win['Order_ID'] * 100)
region_win['Win Rate'].sort_values(ascending=False).plot(kind='bar', ax=axes[1, 1], 
                                                           color='#1abc9c', edgecolor='black')
axes[1, 1].set_title('Win Rate by Region', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Region')
axes[1, 1].set_ylabel('Win Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].axhline(y=df_clean['Is_Won'].mean()*100, color='red', linestyle='--', 
                   linewidth=2, label='Overall Avg')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 3.4 Customer Segment Analysis

In [ ]:
# Customer segment analysis
print("CUSTOMER SEGMENT ANALYSIS")
print("="*60)

segment_analysis = df_clean.groupby('Customer_Segment').agg({
    'Revenue': ['sum', 'mean'],
    'Units_Sold': 'sum',
    'Order_ID': 'count',
    'Is_Won': 'sum'
}).round(2)

segment_analysis.columns = ['Total Revenue', 'Avg Revenue', 'Units Sold', 'Total Orders', 'Won Deals']
segment_analysis['Win Rate %'] = (segment_analysis['Won Deals'] / segment_analysis['Total Orders'] * 100).round(2)
segment_analysis = segment_analysis.sort_values('Total Revenue', ascending=False)

print(segment_analysis)

In [ ]:
# Customer segment visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Customer Segment Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Revenue by Customer Segment
segment_revenue = df_clean.groupby('Customer_Segment')['Revenue'].sum().sort_values(ascending=False)
colors_segment = ['#e74c3c', '#3498db', '#2ecc71']
axes[0, 0].bar(segment_revenue.index, segment_revenue.values, color=colors_segment, edgecolor='black')
axes[0, 0].set_title('Total Revenue by Customer Segment', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Customer Segment')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Average Deal Size by Segment
segment_avg = df_clean.groupby('Customer_Segment')['Deal_Size'].mean().sort_values(ascending=False)
axes[0, 1].bar(segment_avg.index, segment_avg.values, color='#9b59b6', edgecolor='black')
axes[0, 1].set_title('Average Deal Size by Customer Segment', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Customer Segment')
axes[0, 1].set_ylabel('Average Deal Size ($)')
axes[0, 1].tick_params(axis='x', rotation=0)

# 3. Customer Segment Distribution
segment_counts = df_clean['Customer_Segment'].value_counts()
axes[1, 0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%',
               colors=colors_segment, startangle=90)
axes[1, 0].set_title('Customer Segment Distribution', fontsize=12, fontweight='bold')

# 4. Win Rate by Customer Segment
segment_win = df_clean.groupby('Customer_Segment').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
segment_win['Win Rate'] = (segment_win['Is_Won'] / segment_win['Order_ID'] * 100)
segment_win['Win Rate'].sort_values(ascending=False).plot(kind='bar', ax=axes[1, 1], 
                                                            color='#f39c12', edgecolor='black')
axes[1, 1].set_title('Win Rate by Customer Segment', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Customer Segment')
axes[1, 1].set_ylabel('Win Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].axhline(y=df_clean['Is_Won'].mean()*100, color='red', linestyle='--', 
                   linewidth=2, label='Overall Avg')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 3.5 Sales Team Performance

In [ ]:
# Team performance analysis
print("TEAM PERFORMANCE ANALYSIS")
print("="*60)

team_analysis = df_clean.groupby('Team').agg({
    'Revenue': 'sum',
    'Target': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count',
    'Is_Won': 'sum'
}).round(2)

team_analysis.columns = ['Total Revenue', 'Total Target', 'Units Sold', 'Total Orders', 'Won Deals']
team_analysis['Win Rate %'] = (team_analysis['Won Deals'] / team_analysis['Total Orders'] * 100).round(2)
team_analysis['Achievement %'] = (team_analysis['Total Revenue'] / team_analysis['Total Target'] * 100).round(2)
team_analysis = team_analysis.sort_values('Total Revenue', ascending=False)

print(team_analysis)

In [ ]:
# Sales rep performance analysis
print("\nTOP SALES REPRESENTATIVES")
print("="*60)

rep_analysis = df_clean.groupby('Sales_Rep').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum',
    'Order_ID': 'count',
    'Is_Won': 'sum'
}).round(2)

rep_analysis.columns = ['Total Revenue', 'Units Sold', 'Total Orders', 'Won Deals']
rep_analysis['Win Rate %'] = (rep_analysis['Won Deals'] / rep_analysis['Total Orders'] * 100).round(2)
rep_analysis = rep_analysis.sort_values('Total Revenue', ascending=False)

print(rep_analysis)

In [ ]:
# Sales team visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sales Team Performance Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Revenue by Team
team_revenue = df_clean.groupby('Team')['Revenue'].sum().sort_values(ascending=False)
colors_team = ['#e74c3c', '#3498db', '#2ecc71']
axes[0, 0].bar(team_revenue.index, team_revenue.values, color=colors_team, edgecolor='black')
axes[0, 0].set_title('Total Revenue by Team', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Team')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Revenue by Sales Rep
rep_revenue = df_clean.groupby('Sales_Rep')['Revenue'].sum().sort_values(ascending=True)
rep_revenue.plot(kind='barh', ax=axes[0, 1], color='#9b59b6')
axes[0, 1].set_title('Total Revenue by Sales Representative', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Revenue ($)')
axes[0, 1].set_ylabel('Sales Rep')

# 3. Team Achievement Rate (Revenue vs Target)
team_target = df_clean.groupby('Team')[['Revenue', 'Target']].sum()
team_target.plot(kind='bar', ax=axes[1, 0], color=['#2ecc71', '#e67e22'], edgecolor='black')
axes[1, 0].set_title('Team Revenue vs Target', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Team')
axes[1, 0].set_ylabel('Amount ($)')
axes[1, 0].legend(['Revenue', 'Target'])
axes[1, 0].tick_params(axis='x', rotation=0)

# 4. Win Rate by Sales Rep
rep_win = df_clean.groupby('Sales_Rep').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
rep_win['Win Rate'] = (rep_win['Is_Won'] / rep_win['Order_ID'] * 100)
rep_win['Win Rate'].sort_values(ascending=False).plot(kind='bar', ax=axes[1, 1], 
                                                        color='#f39c12', edgecolor='black')
axes[1, 1].set_title('Win Rate by Sales Representative', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Sales Rep')
axes[1, 1].set_ylabel('Win Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].axhline(y=df_clean['Is_Won'].mean()*100, color='red', linestyle='--', 
                   linewidth=2, label='Overall Avg')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

### 3.6 Time-Based Analysis

In [ ]:
# Time-based visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Time-Based Sales Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Daily Revenue Trend
daily_revenue = df_clean.groupby('Order_Date')['Revenue'].sum()
axes[0, 0].plot(daily_revenue.index, daily_revenue.values, linewidth=1.5, color='#3498db', alpha=0.7)
axes[0, 0].set_title('Daily Revenue Trend', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].grid(True, alpha=0.3)

# 2. Monthly Revenue Comparison
monthly_revenue = df_clean.groupby('Month_Name')['Revenue'].sum()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_revenue = monthly_revenue.reindex([m for m in month_order if m in monthly_revenue.index])
axes[0, 1].bar(range(len(monthly_revenue)), monthly_revenue.values, color='#2ecc71', edgecolor='black')
axes[0, 1].set_xticks(range(len(monthly_revenue)))
axes[0, 1].set_xticklabels(monthly_revenue.index, rotation=45, ha='right')
axes[0, 1].set_title('Monthly Revenue', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Revenue ($)')

# 3. Quarterly Performance
quarterly_revenue = df_clean.groupby('Quarter')['Revenue'].sum()
axes[1, 0].bar(quarterly_revenue.index, quarterly_revenue.values, color='#e74c3c', edgecolor='black')
axes[1, 0].set_title('Quarterly Revenue', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Quarter')
axes[1, 0].set_ylabel('Revenue ($)')
axes[1, 0].set_xticks(quarterly_revenue.index)

# 4. Revenue by Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_revenue = df_clean.groupby('Day_of_Week')['Revenue'].sum()
day_revenue = day_revenue.reindex([d for d in day_order if d in day_revenue.index])
axes[1, 1].bar(range(len(day_revenue)), day_revenue.values, color='#9b59b6', edgecolor='black')
axes[1, 1].set_xticks(range(len(day_revenue)))
axes[1, 1].set_xticklabels(day_revenue.index, rotation=45, ha='right')
axes[1, 1].set_title('Revenue by Day of Week', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Revenue ($)')

plt.tight_layout()
plt.show()

### 3.7 Advanced Correlation Analysis

In [ ]:
# Correlation heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Correlation Analysis', fontsize=16, fontweight='bold')

# Select numerical columns for correlation
numerical_cols = ['Units_Sold', 'Revenue', 'Target', 'Deal_Size', 'Achievement_Rate']
correlation_matrix = df_clean[numerical_cols].corr()

# 1. Full correlation heatmap
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, ax=axes[0], cbar_kws={'label': 'Correlation'})
axes[0].set_title('Correlation Matrix - Key Metrics', fontsize=12, fontweight='bold')

# 2. Revenue vs Units Sold scatter plot
won_deals = df_clean[df_clean['Stage'] == 'Won']
axes[1].scatter(won_deals['Units_Sold'], won_deals['Revenue'], alpha=0.5, color='#3498db')
axes[1].set_title('Revenue vs Units Sold (Won Deals)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Units Sold')
axes[1].set_ylabel('Revenue ($)')
axes[1].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(won_deals['Units_Sold'], won_deals['Revenue'], 1)
p = np.poly1d(z)
axes[1].plot(won_deals['Units_Sold'].sort_values(), 
             p(won_deals['Units_Sold'].sort_values()), 
             "r--", linewidth=2, label='Trend Line')
axes[1].legend()

plt.tight_layout()
plt.show()

### 3.8 Distribution Analysis

In [ ]:
# Distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribution Analysis', fontsize=16, fontweight='bold', y=0.995)

# 1. Revenue Distribution (Won Deals)
won_revenue = df_clean[df_clean['Stage'] == 'Won']['Revenue']
axes[0, 0].hist(won_revenue, bins=30, color='#2ecc71', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(won_revenue.mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: ${won_revenue.mean():,.0f}')
axes[0, 0].axvline(won_revenue.median(), color='blue', linestyle='--', linewidth=2, 
                   label=f'Median: ${won_revenue.median():,.0f}')
axes[0, 0].set_title('Revenue Distribution (Won Deals)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Revenue ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# 2. Deal Size Distribution
axes[0, 1].hist(df_clean['Deal_Size'], bins=30, color='#3498db', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(df_clean['Deal_Size'].mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: ${df_clean["Deal_Size"].mean():,.0f}')
axes[0, 1].axvline(df_clean['Deal_Size'].median(), color='blue', linestyle='--', linewidth=2, 
                   label=f'Median: ${df_clean["Deal_Size"].median():,.0f}')
axes[0, 1].set_title('Deal Size Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Deal Size ($)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()

# 3. Box plot - Revenue by Stage
stage_data = [df_clean[df_clean['Stage'] == stage]['Revenue'] for stage in df_clean['Stage'].unique()]
bp = axes[1, 0].boxplot(stage_data, labels=df_clean['Stage'].unique(), patch_artist=True)
for patch, color in zip(bp['boxes'], ['#2ecc71', '#e74c3c', '#f39c12']):
    patch.set_facecolor(color)
axes[1, 0].set_title('Revenue Distribution by Deal Stage', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Deal Stage')
axes[1, 0].set_ylabel('Revenue ($)')
axes[1, 0].grid(True, alpha=0.3)

# 4. Achievement Rate Distribution
achievement_data = df_clean[df_clean['Achievement_Rate'] > 0]['Achievement_Rate']
axes[1, 1].hist(achievement_data, bins=30, color='#9b59b6', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(100, color='red', linestyle='--', linewidth=2, label='Target (100%)')
axes[1, 1].axvline(achievement_data.mean(), color='blue', linestyle='--', linewidth=2, 
                   label=f'Mean: {achievement_data.mean():.1f}%')
axes[1, 1].set_title('Achievement Rate Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Achievement Rate (%)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 4. Insights Report

Write a short report summarizing key findings and trends.

### Executive Summary: GlobalMart Sales Analysis

#### Key Performance Indicators

In [ ]:
# Calculate key insights
total_revenue = df_clean['Revenue'].sum()
total_target = df_clean['Target'].sum()
overall_achievement = (total_revenue / total_target * 100)
win_rate = (df_clean['Is_Won'].sum() / len(df_clean) * 100)
avg_deal_size = df_clean[df_clean['Stage']=='Won']['Deal_Size'].mean()
total_units = df_clean['Units_Sold'].sum()

# Top performers
top_category = df_clean.groupby('Product_Category')['Revenue'].sum().idxmax()
top_category_revenue = df_clean.groupby('Product_Category')['Revenue'].sum().max()
top_region = df_clean.groupby('Region')['Revenue'].sum().idxmax()
top_region_revenue = df_clean.groupby('Region')['Revenue'].sum().max()
top_segment = df_clean.groupby('Customer_Segment')['Revenue'].sum().idxmax()
top_rep = df_clean.groupby('Sales_Rep')['Revenue'].sum().idxmax()
top_rep_revenue = df_clean.groupby('Sales_Rep')['Revenue'].sum().max()

print("GLOBALMART SALES INSIGHTS REPORT")
print("="*60)
print("\n1. OVERALL PERFORMANCE")
print("-" * 60)
print(f"Total Revenue Generated: ${total_revenue:,.2f}")
print(f"Total Sales Target: ${total_target:,.2f}")
print(f"Overall Achievement Rate: {overall_achievement:.2f}%")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Average Deal Size (Won): ${avg_deal_size:,.2f}")
print(f"Total Units Sold: {total_units:,}")

print("\n2. TOP PERFORMERS")
print("-" * 60)
print(f"Best Product Category: {top_category} (${top_category_revenue:,.2f})")
print(f"Best Region: {top_region} (${top_region_revenue:,.2f})")
print(f"Best Customer Segment: {top_segment}")
print(f"Top Sales Representative: {top_rep} (${top_rep_revenue:,.2f})")

print("\n3. KEY INSIGHTS")
print("-" * 60)

# Product insights
product_win_rates = df_clean.groupby('Product_Category').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
product_win_rates['Win Rate'] = (product_win_rates['Is_Won'] / product_win_rates['Order_ID'] * 100)
best_win_product = product_win_rates['Win Rate'].idxmax()
best_win_rate = product_win_rates['Win Rate'].max()

print(f"\nProduct Performance:")
print(f"  - {top_category} generates the highest revenue")
print(f"  - {best_win_product} has the best win rate at {best_win_rate:.2f}%")

# Regional insights
region_win_rates = df_clean.groupby('Region').agg({
    'Is_Won': 'sum',
    'Order_ID': 'count'
})
region_win_rates['Win Rate'] = (region_win_rates['Is_Won'] / region_win_rates['Order_ID'] * 100)
best_win_region = region_win_rates['Win Rate'].idxmax()

print(f"\nRegional Performance:")
print(f"  - {top_region} leads in total revenue generation")
print(f"  - {best_win_region} region shows the highest conversion rate")

# Customer segment insights
segment_avg_deal = df_clean.groupby('Customer_Segment')['Deal_Size'].mean()
best_segment_deal = segment_avg_deal.idxmax()
best_segment_value = segment_avg_deal.max()

print(f"\nCustomer Segment Performance:")
print(f"  - {top_segment} segment contributes the most revenue")
print(f"  - {best_segment_deal} segment has highest average deal size (${best_segment_value:,.2f})")

# Time-based insights
monthly_revenue = df_clean.groupby('Month')['Revenue'].sum()
best_month = df_clean.groupby('Month_Name')['Revenue'].sum().idxmax()
quarterly_growth = df_clean.groupby('Quarter')['Revenue'].sum()

print(f"\nTemporal Patterns:")
print(f"  - {best_month} was the strongest performing month")
print(f"  - Revenue shows variation across quarters")

print("\n4. RECOMMENDATIONS")
print("-" * 60)
print(f"  - Focus on expanding {top_category} product line in high-performing regions")
print(f"  - Replicate successful strategies from {best_win_region} region to other areas")
print(f"  - Invest more resources in {top_segment} customer segment")
print(f"  - Share best practices from top performer {top_rep} with the team")
print(f"  - Analyze and address factors contributing to lost deals (improve {100-win_rate:.2f}% loss rate)")
print(f"  - Develop targeted campaigns for converting opportunities to won deals")

print("\n" + "="*60)

## Conclusion

This comprehensive EDA has provided deep insights into GlobalMart's sales performance across multiple dimensions including products, geography, customer segments, sales teams, and time periods. The analysis reveals clear patterns and opportunities for optimization in the sales strategy.

### Key Takeaways:

1. The company maintains a healthy win rate and shows strong performance in specific categories
2. Geographic expansion opportunities exist in high-performing regions
3. Customer segmentation analysis reveals distinct purchasing patterns
4. Sales team performance varies, indicating potential for knowledge sharing
5. Temporal analysis shows seasonal patterns that can inform inventory and marketing strategies

### Next Steps:

- Implement targeted strategies for underperforming segments
- Conduct deeper analysis into lost deals to identify improvement areas
- Develop predictive models for sales forecasting
- Create real-time dashboards for ongoing performance monitoring

---

**Analysis Complete**